In [1]:
print("hi")

hi


### Import requests

In [10]:
!pip install requests --quiet
!pip install pandas --quiet
!pip install matplotlib --quiet
!pip install seaborn --quiet

import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [11]:
CLIENT_ID = '586ecea45142471cb181007f2ffe630b'
CLIENT_SECRET = '187ee73885934d9d86a78d4965b7d6d7'

### Grab access Token

In [12]:
auth_response = requests.post('https://accounts.spotify.com/api/token',
    {'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET}).json()
access_token = auth_response['access_token']
HEADERS = {'Authorization': 'Bearer {token}'.format(token=access_token)}

In [13]:
def get_listening_history(token, before, limit):
    # get the endpoint url
    url = 'https://api.spotify.com/v1/me/player/recently-played'
    
    # Get the Authorization header for the request
    headers = {'Authorization': f'Bearer {token}'}
    
    # setup the query
    query = f"?before={before}&limit={limit}"
    
    # et the query  url
    query_url = url + query
    
     #Send a GET request to the Spotify API with the query URL and headers
    result = requests.get(url, headers=headers)
    result.json()['items']

In [15]:
import datetime

# Function to get the start and end timestamps for a specific day
def get_day_window(date):
    date_obj = datetime.datetime.strptime(date, '%Y-%m-%d') # Convert the date string to a datetime object
    
    # Get the start timestamp for the day (00:00:00)
    start_timestamp = date_obj.replace(hour=0, minute=0, second=0).isoformat() + 'Z'
    
    # Get the end timestamp for the day (23:59:59)
    end_timestamp = date_obj.replace(hour=23, minute=59, second=59).isoformat() + 'Z'
    
    return start_timestamp, end_timestamp

# Example usage
date_to_fetch = '2024-04-21'  # Date for which you want to fetch the listening history
start_time, end_time = get_day_window(date_to_fetch)

print("Start Timestamp:", start_time)
print("End Timestamp:", end_time)


Start Timestamp: 2024-04-21T00:00:00Z
End Timestamp: 2024-04-21T23:59:59Z


In [17]:
# Fetching listening history for the specified day
listening_history = get_listening_history(access_token, before=end_time, limit=50)

if listening_history:
    print("Tracks listened to on", date_to_fetch + ":")
    for track in listening_history:
        print(track['track']['name'])  # Printing the name of each track

Error: 403


KeyError: 'access_token'